In [1]:
import pandas as pd
import warnings
from sklearn import metrics
import processing
import time
import os
from tqdm import tqdm
from torch.optim import Adam
from torch import nn
import numpy as np
import sys
sys.path.append("D:/Experiment")
import torch
from MyKu import processing
from MyKu import MyXLM_Base
warnings.filterwarnings("ignore")


In [2]:

# 训练准备阶段，设置超参数和全局变量
file_name = 'readme_taskB.md'
batch_size = 32
num_epoch = 20  # 训练轮次
check_step = 1  # 用以训练中途对模型进行检验：每check_step个epoch进行一次测试和保存模型

learning_rate = 1e-5  # 优化器的学习率

# 获取训练、测试数据、分类类别总数
en_train_data, en_test_data, de_train_data, de_test_data, hi_train_data, hi_test_data = processing.load_hasoc2020_for_taskB()
categories = 3

en_train_iter, en_test_iter = MyXLM_Base.load_xlm_data(en_train_data, en_test_data, batch_size)
de_train_iter, de_test_iter = MyXLM_Base.load_xlm_data(de_train_data, de_test_data, batch_size)
hi_train_iter, hi_test_iter = MyXLM_Base.load_xlm_data(hi_train_data, hi_test_data, batch_size)
#固定写法，可以牢记，cuda代表Gpu
# torch.cuda.is_available()可以查看当前Gpu是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练模型，因为这里是英文数据集，需要用在英文上的预训练模型：bert-base-uncased
# uncased指该预训练模型对应的词表不区分字母的大小写
# 详情可了解：https://huggingface.co/bert-base-uncased
pretrained_model_name = 'xlm-roberta-base'
# 创建模型 BertSST2Model
model = MyXLM_Base.MyXlmModel(categories, pretrained_model_name)
# 固定写法，将模型加载到device上，
# 如果是GPU上运行，此时可以观察到GPU的显存增加
model.to(device)

# 训练过程
# Adam是最近较为常用的优化器，详情可查看：https://www.jianshu.com/p/aebcaf8af76e
optimizer = Adam(model.parameters(), learning_rate)  # 使用Adam优化器
loss = nn.CrossEntropyLoss()  # 使用crossentropy作为二分类任务的损失函数

# 记录当前训练时间，用以记录日志和存储
timestamp = time.strftime("%m_%d_%H_%M", time.localtime())


In [4]:
# TaskB

file_name = 'readme_taskB.md'
model_save_path = 'D:/Experiment_models_save/twitter-xlm-roberta-base-sentiment/hasoc2020/'
name = 'base_taskB.pth'
en_temp_best = 0.7
de_temp_best = 0.7
hi_temp_best = 0.5
for epoch in range(1, num_epoch + 1):
    MyXLM_Base.train(model, en_train_iter, device, optimizer, loss, epoch)
    MyXLM_Base.train(model, de_train_iter, device, optimizer, loss, epoch)
    MyXLM_Base.train(model, hi_train_iter, device, optimizer, loss, epoch)
    en_acc_score = MyXLM_Base.test(model, en_test_iter, device, epoch, file_name)
    de_acc_score = MyXLM_Base.test(model, de_test_iter, device, epoch, file_name)
    hi_acc_score = MyXLM_Base.test(model, hi_test_iter, device, epoch, file_name)
    print('\n\n')
    if en_acc_score > en_temp_best and de_acc_score > de_temp_best and hi_acc_score > hi_temp_best:
        en_temp_best, de_temp_best, hi_temp_best = en_acc_score, de_acc_score, hi_acc_score
        MyXLM_Base.save_pretrained(model, model_save_path, name)
        print(f'best en_acc_socre : {en_acc_score}, best de_acc_score : {de_acc_score}, best hi_acc_score : {hi_acc_score}')
print(f'best en_acc_socre : {en_temp_best}, best de_acc_score : {de_temp_best}, best de_acc_score : {hi_temp_best}')


Testing: 100%|██████████| 807/807 [00:08<00:00, 98.01it/s] 


[[533  15  32]
 [112  32  25]
 [ 29   4  25]]
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       580
           1       0.63      0.19      0.29       169
           2       0.30      0.43      0.36        58

    accuracy                           0.73       807
   macro avg       0.57      0.51      0.50       807
weighted avg       0.72      0.73      0.70       807

Acc : 0.7311028500619579	 F1: 0.49937723095617836


Testing: 100%|██████████| 300/300 [00:02<00:00, 101.02it/s]


[[136  12  29]
 [ 26  20  20]
 [  3   2  52]]
              precision    recall  f1-score   support

           0       0.82      0.77      0.80       177
           1       0.59      0.30      0.40        66
           2       0.51      0.91      0.66        57

    accuracy                           0.69       300
   macro avg       0.64      0.66      0.62       300
weighted avg       0.71      0.69      0.68       300

Acc : 0.6933333333333334	 F1: 0.6178498285093889


Testing: 100%|██████████| 367/367 [00:04<00:00, 84.58it/s]


...

[[548  25   7]
 [115  52   2]
 [ 34   9  15]]
              precision    recall  f1-score   support

           0       0.79      0.94      0.86       580
           1       0.60      0.31      0.41       169
           2       0.62      0.26      0.37        58

    accuracy                           0.76       807
   macro avg       0.67      0.50      0.54       807
weighted avg       0.74      0.76      0.73       807

Acc : 0.7620817843866171	 F1: 0.5439861154334976


Testing: 100%|██████████| 300/300 [00:03<00:00, 95.39it/s] 


[[149  21   7]
 [ 29  29   8]
 [  7   5  45]]
              precision    recall  f1-score   support

           0       0.81      0.84      0.82       177
           1       0.53      0.44      0.48        66
           2       0.75      0.79      0.77        57

    accuracy                           0.74       300
   macro avg       0.69      0.69      0.69       300
weighted avg       0.73      0.74      0.74       300

Acc : 0.7433333333333333	 F1: 0.6905913440318262


Testing: 100%|██████████| 367/367 [00:03<00:00, 92.69it/s]

[[ 28  34   1]
 [ 22 141  28]
 [ 14  59  40]]
              precision    recall  f1-score   support

           0       0.44      0.44      0.44        63
           1       0.60      0.74      0.66       191
           2       0.58      0.35      0.44       113

    accuracy                           0.57       367
   macro avg       0.54      0.51      0.51       367
weighted avg       0.57      0.57      0.56       367

Acc : 0.5694822888283378	 F1: 0.5146782444049697



best en_acc_socre : 0.7633209417596035, best de_acc_score : 0.7666666666666667, best de_acc_score : 0.5776566757493188
